# Capitolul 7: Crearea aplicațiilor de chat
## Ghid rapid pentru API-ul Github Models

Acest notebook este adaptat din [Repository-ul de exemple Azure OpenAI](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst), care include notebook-uri ce accesează serviciile [Azure OpenAI](notebook-azure-openai.ipynb).


# Prezentare generală  
„Modelele lingvistice mari sunt funcții care transformă textul în text. Pornind de la un șir de text introdus, un model lingvistic mare încearcă să prezică ce text va urma”(1). Acest notebook de tip „quickstart” îi va familiariza pe utilizatori cu conceptele de bază ale LLM, cerințele principale ale pachetului pentru a începe cu AML, o introducere ușoară în designul prompturilor și câteva exemple scurte pentru diferite scenarii de utilizare.


## Cuprins  

[Prezentare generală](../../../../07-building-chat-applications/python)  
[Cum să folosești serviciul OpenAI](../../../../07-building-chat-applications/python)  
[1. Crearea serviciului OpenAI](../../../../07-building-chat-applications/python)  
[2. Instalare](../../../../07-building-chat-applications/python)    
[3. Date de autentificare](../../../../07-building-chat-applications/python)  

[Utilizări](../../../../07-building-chat-applications/python)    
[1. Rezumă textul](../../../../07-building-chat-applications/python)  
[2. Clasifică textul](../../../../07-building-chat-applications/python)  
[3. Generează nume noi de produse](../../../../07-building-chat-applications/python)  
[4. Ajustează fin un clasificator](../../../../07-building-chat-applications/python)  

[Referințe](../../../../07-building-chat-applications/python)


### Creează primul tău prompt  
Acest exercițiu scurt oferă o introducere de bază pentru trimiterea de prompturi către un model în Github Models pentru o sarcină simplă de „rezumare”.


**Pași**:  
1. Instalează biblioteca `azure-ai-inference` în mediul tău python, dacă nu ai făcut-o deja.  
2. Încarcă bibliotecile standard de ajutor și configurează acreditările pentru Github Models.  
3. Alege un model pentru sarcina ta  
4. Creează un prompt simplu pentru model  
5. Trimite cererea ta către API-ul modelului!


### 1. Instalează `azure-ai-inference`


In [ ]:
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

### 3. Găsirea modelului potrivit  
Modelele GPT-3.5-turbo sau GPT-4 pot înțelege și genera limbaj natural.


In [ ]:
# Select the General Purpose curie model for text
model_name = "gpt-4o"

## 4. Proiectarea prompturilor  

„Magia modelelor lingvistice mari constă în faptul că, fiind antrenate să minimizeze această eroare de predicție pe cantități uriașe de text, modelele ajung să învețe concepte utile pentru aceste predicții. De exemplu, ele învață concepte precum”(1):

* cum se scrie corect
* cum funcționează gramatica
* cum să reformuleze
* cum să răspundă la întrebări
* cum să poarte o conversație
* cum să scrie în mai multe limbi
* cum să scrie cod
* etc.

#### Cum controlezi un model lingvistic mare  
„Dintre toate intrările pentru un model lingvistic mare, de departe cea mai influentă este promptul textului”(1).

Modelele lingvistice mari pot fi stimulate să genereze rezultate în câteva moduri:

Instrucțiune: Spune-i modelului ce vrei
Completare: Induci modelul să continue începutul a ceea ce dorești
Demonstrație: Arată-i modelului ce vrei, fie cu:
Câteva exemple în prompt
Câteva sute sau mii de exemple într-un set de date de antrenament pentru fine-tuning”



#### Există trei reguli de bază pentru crearea prompturilor:

**Arată și explică**. Fă clar ce vrei, fie prin instrucțiuni, exemple sau o combinație a celor două. Dacă vrei ca modelul să ordoneze o listă de elemente alfabetic sau să clasifice un paragraf după sentiment, arată-i exact asta.

**Furnizează date de calitate**. Dacă încerci să construiești un clasificator sau să faci modelul să urmeze un anumit tipar, asigură-te că ai suficiente exemple. Verifică-ți exemplele — modelul este de obicei destul de inteligent să treacă peste greșeli de ortografie și să-ți dea un răspuns, dar ar putea presupune că acestea sunt intenționate și asta poate influența rezultatul.

**Verifică setările.** Setările temperature și top_p controlează cât de determinist este modelul în generarea unui răspuns. Dacă îi ceri un răspuns unde există un singur răspuns corect, ar trebui să setezi aceste valori mai jos. Dacă vrei răspunsuri mai diverse, le poți seta mai sus. Cea mai frecventă greșeală pe care o fac oamenii cu aceste setări este să creadă că ele controlează „inteligența” sau „creativitatea” modelului.


Sursa: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Rezumă textul  
#### Provocare  
Rezumă un text adăugând 'tl;dr:' la finalul unui pasaj. Observă cum modelul înțelege să realizeze mai multe sarcini fără instrucțiuni suplimentare. Poți experimenta cu solicitări mai descriptive decât tl;dr pentru a modifica comportamentul modelului și a personaliza rezumatul primit(3).  

Cercetări recente au arătat îmbunătățiri semnificative în multe sarcini și benchmark-uri NLP prin pre-antrenarea pe un corpus mare de texte, urmată de ajustarea fină pe o sarcină specifică. Deși această metodă are o arhitectură, în general, independentă de sarcină, tot necesită seturi de date specifice pentru ajustare fină, cu mii sau zeci de mii de exemple. În schimb, oamenii pot realiza, de obicei, o nouă sarcină lingvistică doar din câteva exemple sau instrucțiuni simple – lucru cu care sistemele NLP actuale încă se confruntă. Aici arătăm că extinderea modelelor lingvistice îmbunătățește semnificativ performanța generală, cu puține exemple, uneori ajungând chiar să concureze cu abordările anterioare de ajustare fină de ultimă generație.



Tl;dr


# Exerciții pentru mai multe scenarii de utilizare  
1. Rezumă textul  
2. Clasifică textul  
3. Generează nume noi de produse


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Clasifică Textul  
#### Provocare  
Clasifică elementele în categorii furnizate în momentul inferenței. În exemplul de mai jos, atât categoriile cât și textul de clasificat sunt incluse în prompt (*playground_reference).

Solicitare client: Bună ziua, una dintre tastele de la tastatura laptopului meu s-a stricat recent și am nevoie de una nouă:

Categorie clasificată:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Generează Nume Noi de Produse
#### Provocare
Creează nume de produse pornind de la cuvinte exemplu. În acest exercițiu includem în prompt informații despre produsul pentru care dorim să generăm nume. Oferim și un exemplu similar pentru a arăta modelul pe care îl dorim. De asemenea, am setat o valoare mare pentru temperatură pentru a crește gradul de inovație și răspunsuri mai creative.

Descriere produs: Un aparat de făcut milkshake-uri acasă  
Cuvinte cheie: rapid, sănătos, compact.  
Nume de produse: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Descriere produs: O pereche de pantofi care se potrivește oricărei mărimi de picior.  
Cuvinte cheie: adaptabil, potrivire, omni-fit.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# Referințe  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [Exemple OpenAI Studio](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [Cele mai bune practici pentru fine-tuning GPT-3 pentru clasificarea textului](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# Pentru mai mult ajutor  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)


# Contribuitori
* [Chew-Yean Yam](https://www.linkedin.com/in/cyyam/)



---

**Declinarea responsabilității**:
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși depunem eforturi pentru acuratețe, vă rugăm să rețineți că traducerile automate pot conține erori sau inexactități. Documentul original, în limba sa nativă, trebuie considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de oameni. Nu ne asumăm răspunderea pentru eventuale neînțelegeri sau interpretări greșite care pot apărea în urma utilizării acestei traduceri.
